<a href="https://colab.research.google.com/github/LeyloJohnny/Computational-Neuroscience/blob/main/Cross_validation_HMM_no_overlap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#get data from google drive
!gdown --id 1qblO7uQCJ6KKclqS-rhvXfQ7ccWcmxaA
#unzip data
!unzip 'NewData.zip'

Downloading...
From: https://drive.google.com/uc?id=1qblO7uQCJ6KKclqS-rhvXfQ7ccWcmxaA
To: /content/NewData.zip
127MB [00:01, 123MB/s]
Archive:  NewData.zip
  inflating: Cluster.pkl             
  inflating: BN.pkl                  
  inflating: SFA.pkl                 


In [ ]:
!unzip 'NewData.zip'

Archive:  NewData.zip
replace Cluster.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pickle

#create a list of model name strings
#models = ['BN.pkl', "SFA.pkl", "Cluster.pkl"]

#create a list of model names
#model_names = ["Balanced Network", "Clustered with SFA", "Clustered wo SFA"]

#create a list of model name strings
models = ['new_BN.pkl', "new_SFA.pkl", "new_Cluster.pkl"]

#modes = ["E", "I_and_E"]

#create a list of model names
model_names = ["Balanced Network", "Clustered with SFA", "Clustered wo SFA"]

#create lists to store model data
models_x = []
models_y = []

#for all models
for i in range(len(models)):

  #open pickle file
  with open(models[i], 'rb') as f:
    #get x (actual data: time * neuron id)
    x = pickle.load(f)
    #get y (information on data, such as inhibitory/excitatory neurons, clusters, etc.)
    y = pickle.load(f)
    #add x and y to the models lists
    models_x.append(x)
    models_y.append(y)

In [ ]:
import numpy as np
import pandas as pd

time_range_models = []

time_range = 10000
starting_time = 0

for model in range(len(models)):
  #set condition: returns True for time <= 5 s
  condition = [True if i > starting_time and i <= time_range + starting_time else False for i in models_x[model][0]]

  #extract new x and y values for array based on condition
  new_x = np.extract(condition, models_x[model][0])
  new_y = np.extract(condition, models_x[model][1])

  new = [new_x, new_y]

  time_range_models.append(new)

In [ ]:
import matplotlib.pyplot as plt

one_cluster = []

neuron_ID = 0

for model in range(len(models)):

  condition = [True if i > neuron_ID and i <= neuron_ID+4000 else False for i in time_range_models[model][1]]

  #extract new x and y values for array based on condition
  new_x = np.extract(condition, time_range_models[model][0])
  new_y = np.extract(condition, time_range_models[model][1])

  #create figure
  figure = plt.figure(dpi=100)
  #create subplot
  ax = figure.add_subplot(1,1,1)
  #plot both dimensions of x (time, neuron id)
  plt.plot(new_x,new_y,'.k',markersize = 0.8) 

  #label the plot
  ax.set_xlabel('Time in ms', fontsize = 10)
  ax.set_ylabel('Neuron ID', fontsize = 10)
  ax.set_title(model_names[model], fontsize = 15)

  new = [new_x, new_y]

  one_cluster.append(new)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#define a time window (in milliseconds)
window = 100

file_name = "spike_count_matrix_tr_" + str(time_range) + "_tw_" + str(window)

#calculate number of intervals based on recorded time span / time window
intervals = int(np.max(time_range_models[0][0]+ 1-starting_time)/window)

#define which neurons to focus on (here: only excitatory neurons 0 - 4000)
Nneurons = int(np.max(time_range_models[0][1])+1) -1000

#how many intervals do we have?
print("We have " + str(intervals) + " intervals, each " + str(window) + "ms long. \n")

spike_count_matrix_average = np.zeros((len(models), intervals, Nneurons))

#for every model
for i in range(len(models)):
  #get the data of the current model
  current_model_x = np.array(time_range_models[i])

  #create a zeros array with size time * neurons (5000*4000)
  spike_counts = np.zeros((intervals, Nneurons))

  #for every interval
  for j in range(intervals):

    #get the indices of time points within the current interval
    interval_time_points = np.where(np.logical_and(window*j + starting_time <= current_model_x[0], current_model_x[0] < window*(j+1) + starting_time))

    #get data indexed by current interval
    interval_data = current_model_x[1][interval_time_points]
    
    #for every neuron
    for k in range(Nneurons):

        #get the indices of the current neuron within the defined time interval
        current_neuron = np.where(interval_data == k)

        #get the number of spikes based on the amount of neuron indices within the defined time interval
        spikes = current_neuron[0].shape[0]

        #add spikes number to spike count matrix
        spike_counts[j, k] = spikes
  
  #add spike count matrix for every model
  spike_count_matrix_average[i] = spike_counts

#print data
print("The spike count matrix has the shape: " + str(spike_count_matrix_average.shape) + " representing " + 
      str(spike_count_matrix_average.shape[0]) + " models, " + str(spike_count_matrix_average.shape[1]) + " intervals, and " + str(spike_count_matrix_average.shape[2]) + " neurons. \n")
print("Spike count matrix: \n")
print(spike_count_matrix_average)

In [ ]:
#define titles for columns and rows in the dataframe that will be generated from the spike count matrix
cols = ['Neuron{}'.format(int(col+1)) for col in range(Nneurons)] #rows represent the neurons (= features)
rows = ['Interval{}'.format(int(row+1)) for row in range(intervals)] #columns represent the time intervals/time bins (= samples)

#create list for the data to be analyzed for each model
models_data_average = []

#for every model
for i in range(len(models)):
  #create a dataframe from the spike count matrix
  current = spike_count_matrix_average[i]
  data_average = pd.DataFrame(data = current, index = rows, columns = cols)
  #add the dataframe to the list (containing one dataframe per model)
  models_data_average.append(data_average)

#exemplarily show the dataframe for the second model
models_data_average[1].head(intervals)

In [ ]:
model_clusters = []

cluster_length = 80
clusters = int(Nneurons/cluster_length)

for model in range(len(models)):

  all_cluster_df = pd.DataFrame()

  for cluster in range(clusters):
    cols = range(cluster*cluster_length, cluster*cluster_length + cluster_length)
    series = models_data_average[model][models_data_average[1].columns[cols]].mean(axis = 1)
    all_cluster_df.insert(cluster,"cluster" + str(cluster+1),series,True)
  
  model_clusters.append(all_cluster_df)

In [ ]:
all_cluster_df

In [ ]:
 data_array = []
 for model in range(len(models)): 
  new_array = model_clusters[model].to_numpy()
  data_array.append(new_array)

In [ ]:
!pip install git+git://github.com/lindermanlab/ssm
import ssm

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold
import scipy.stats as st
import multiprocessing

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(3)

import ssm
from ssm.util import find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

color_names=['windows blue','red','amber','faded green','dusty purple','orange','steel blue','pink',
             'greyish','mint','clay','light cyan','forest green','pastel purple','salmon','dark brown',
             'lavender','pale green','dark red','gold','dark teal','rust','fuchsia','pale orange',
             'cobalt blue','mahogany','cloudy blue','dark pastel green','dust','electric lime','fresh green','light eggplant']

color = sns.xkcd_palette(color_names)
cmap = gradient_cmap(color)

In [ ]:
obs_dim = 50

#how many subsamples?
parts = 10

#how long is each subsample
part_length = int(model_clusters[1].shape[0]/parts)
time_bins = part_length
print(part_length)

#create list for subsamples
models_subsamples = []

#for all models
for model in range(len(models)):
  #create lists for current part data
  current_part_data = []
  models_parts = []

  #for every sample (sample: part of the whole time range of the data)
  for part in range(parts): 
    #define part time range
    part_start = part*part_length 
    part_end = (part+1)*part_length

    #extract current part data based on defined time range
    current_part = model_clusters[model][part_start:part_end]
    current_part_data.append(current_part.to_numpy())

    #sample = current_part.sample(int(obs_dim), axis = 'columns')

    #for neuron in range(sample.shape[1]):
      #del current_part_data[part][list(sample)[neuron]]
    
    #models_parts.append(sample.to_numpy())

  #models_subsamples.append(models_parts)
  models_subsamples.append(current_part_data)

In [ ]:
from sklearn.model_selection import KFold

Ks = np.arange(2,30,2)

X = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

synthetic_data = models_subsamples[2]

subsamples = len(synthetic_data)

runs = 5

#10 different number of latent states
#5-fold cross-validation
#That's 50 model fits! Thankfully this can be parallelized fairly easily
#max_states = 20
N_iters = 5
ll_training = np.zeros((len(Ks),runs))
ll_heldout = np.zeros((len(Ks),runs))

nTrain = int(0.8 * len(synthetic_data))
nTest = int(0.2 * len(synthetic_data))

hmm_z_ms = np.zeros((len(Ks),time_bins))

#Outer loop over the parameter for which you're doing model selection for
for iS, num_states in enumerate(Ks):

    print("----------------------------" + str(iS) + " ---------------------------")
    #iK = 0
    kf = KFold(n_splits=runs, shuffle = True) 

    for iK, (train_index, test_index) in enumerate(kf.split(X)):

        print("iK: " + str(iK) + " TRAIN:", train_index, "TEST:", test_index)
          
        #Split data
        training_data = [synthetic_data[ii] for ii in train_index]
        test_data = [synthetic_data[ii] for ii in test_index]

        #Create HMM object to fit
        hmm = ssm.HMM(num_states, obs_dim, observations="gaussian")

        #fit on training data
        hmm_lls = hmm.fit(training_data, method="em", num_iters=N_iters)#, init_method="kmeans")
          
        #Compute log-likelihood for each dataset
        ll_training[iS,iK] = hmm.log_probability(training_data)/nTrain
        ll_heldout[iS,iK] = hmm.log_probability(test_data)/nTest

        #iK += 1

In [ ]:
#Plot the log-likelihood of the training and test datasets

fig, axes = plt.subplots(1,2,figsize=(20,7),sharey=False)

#Training data
axes[0].plot(Ks, ll_training,'.')
axes[0].plot(Ks, np.mean(ll_training,axis=1),'-ok')
axes[0].set_title('Training Log-likelihood')
axes[0].set_ylabel('Log-likelihood / data-point')

axes[1].plot(Ks, ll_heldout,'.')
axes[1].plot(Ks, np.mean(ll_heldout,axis=1),'-ok')
axes[1].set_title('Test Log-likelihood')

mean_test_data = np.mean(ll_heldout,axis=1)

n_max = max(mean_test_data)
n_max_index = Ks[np.where(mean_test_data == n_max)]

for ax in axes:
    ax.set_xticks(Ks)
    ax.set_xlabel('Number of States')
    #ax.vlines(n_max_index,*ax.get_ylim(),ls='--',color='k')
